# The Project Aims to Determine the Air Quality by Using Lasso Regression and Random Forest Regressor

In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import urllib
import matplotlib.pyplot as plt
%matplotlib inline

We will be trying to predict the pollution (PM2.5) using:
- temperature ('TEMP')
- pressure ('PRES')
- dew-point temperature ('DEWP')
- precipitation ('RAIN')
- wind direction ('wd')
- wind speed ('WSPM')

In [4]:
urllib.request.urlretrieve('https://drive.google.com/uc?id=1m1g4Xn1wMAGV_EU0Nh1HTI1ogA3-tqJk&export=download', './data.csv')

('./data.csv', <http.client.HTTPMessage at 0x1e626e96f48>)

In [5]:
raw_df = pd.read_csv('data.csv',index_col='No')

#put the columns in a useful order
raw_df = raw_df[['PM2.5', 'year', 'month', 'day', 'hour', 'PM10', 'SO2', 'NO2', 'CO',
       'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'wd', 'WSPM', 'station']]

In [6]:
raw_df.head(5)

,PM2.5,year,month,day,hour,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
No,,,,,,,,,,,,,,,,,
1,9.0,2013,3,1,0,9.0,6.0,17.0,200.0,62.0,0.3,1021.9,-19.0,0.0,WNW,2.0,Wanshouxigong
2,11.0,2013,3,1,1,11.0,7.0,14.0,200.0,66.0,-0.1,1022.4,-19.3,0.0,WNW,4.4,Wanshouxigong
3,8.0,2013,3,1,2,8.0,NaN,16.0,200.0,59.0,-0.6,1022.6,-19.7,0.0,WNW,4.7,Wanshouxigong
4,8.0,2013,3,1,3,8.0,3.0,16.0,NaN,NaN,-0.7,1023.5,-20.9,0.0,NW,2.6,Wanshouxigong
5,8.0,2013,3,1,4,8.0,3.0,NaN,300.0,36.0,-0.9,1024.1,-21.7,0.0,WNW,2.5,Wanshouxigong


Some of the records are missing. We need to handle that before we can easily use the data with most ML tools.

### Removing missing data 

I am going to handle this by dropping those rows which have a NaN in one of these columns: ['PM2.5','hour','TEMP','PRES','DEWP','RAIN','wd','WSPM'].

I will save the result in `nonull_df`.

In [7]:
#using dropna method of pandas
nonull_df = raw_df.dropna(subset= ['PM2.5','hour','TEMP','PRES','DEWP','RAIN','wd','WSPM']) # dropping the rows that contains NaN in the selected columns

Here I am checking if the missing values are correctly removed or not. I have used Pandas `clean_df.isnull().sum()` method to confirm that there are no NaN rows in the columns of my interest:

In [8]:
nonull_df[['PM2.5','hour','TEMP','PRES','DEWP','RAIN','wd','WSPM']].isnull().sum() # Checking the sum of NaN values for all the rows in the selected columns

PM2.5    0
hour     0
TEMP     0
PRES     0
DEWP     0
RAIN     0
wd       0
WSPM     0
dtype: int64

###  Removing unwanted columns 

I am removing the features (columns) that will not be used for the prediction. I have used `nonull_df.drop(list_of_column_names, axis=1)` to do this. I will drop: ['year','month','day','PM10','SO2','NO2','CO','O3','station'].

In [9]:
unwanted_columns = ['year','month','day','PM10','SO2','NO2','CO','O3','station'] # List of unwanted columns
clean_df = nonull_df.drop(unwanted_columns, axis=1) # we are dropping the list of unwanted columns

In [10]:
clean_df.isnull().sum() # rechecking the fact that there are no null values in the selected columns

PM2.5    0
hour     0
TEMP     0
PRES     0
DEWP     0
RAIN     0
wd       0
WSPM     0
dtype: int64

In [11]:
#there should be 34284 rows left in the dataframe, and 8 columns
clean_df.shape #=(34284, 8)

(34284, 8)

###  Splitting the dataset 

Before designing any machine learning model, we need to set aside the test data. We will use the remaining training data for fitting the model. **It is important to remember that the test data has to be set aside before preprocessing**.

Any preprocessing that we do has to be done only on the training data and several key statistics need to be saved for the test stage. Separating the dataset into training and test before any preprocessing has happened help us to recreate the real world scenario where we will deploy our system and for which the data will come without any preprocessing.

Later I will be performing a grid search to select parameter values. To do this I've done cross-validation, but rather than splitting the data into training and validation here, I'll split it later. So for now I'll just split into:

- The training (and validation) set will have 85% of the total observations, 
- The test set, the remaining 15%.

To avoid unwanted correlations connecting the training and test, I will split these by time. So:

- I take the first 85% of the rows from clean_df and put them in train_df, and I take the remaining 15% of the rows and put them in test_df

In [14]:
n = 85
train_size = round(len(clean_df)*n/100) # Calculating the training size as 85% of the total size of the dataset 
#print(train_size)
test_size = round(len(clean_df)-train_size) # taking the remaining as the test size
#print(test_size)

#train_df = clean_df[len()]

In [17]:
train_df = clean_df[:train_size+1] # Putting the first 85% of the values in the train_df
test_df = clean_df[train_size:] # Putting the last 15% of the values in the test_df

To check the sizes are correct, we can use:

In [19]:
len(train_df)/len(clean_df),len(test_df)/len(clean_df) #checking whether the ratio of the elements in the train and test dataset are correct or not

(0.8500175008750438, 0.15001166725002918)

In [20]:
train_df.head(5)

,PM2.5,hour,TEMP,PRES,DEWP,RAIN,wd,WSPM
No,,,,,,,,
1,9.0,0,0.3,1021.9,-19.0,0.0,WNW,2.0
2,11.0,1,-0.1,1022.4,-19.3,0.0,WNW,4.4
3,8.0,2,-0.6,1022.6,-19.7,0.0,WNW,4.7
4,8.0,3,-0.7,1023.5,-20.9,0.0,NW,2.6
5,8.0,4,-0.9,1024.1,-21.7,0.0,WNW,2.5


In [21]:
test_df.head(5)

,PM2.5,hour,TEMP,PRES,DEWP,RAIN,wd,WSPM
No,,,,,,,,
29766,119.0,5,25.9,996.8,24.9,0.0,E,0.5
29767,128.0,6,26.2,997.3,25.2,0.0,E,0.5
29768,116.0,7,26.7,997.6,25.5,0.0,E,0.5
29769,123.0,8,27.4,997.4,25.2,0.0,NE,1.0
29770,148.0,9,27.9,997.5,25.5,0.0,NNE,1.1


# Detour: Lasso Regression

Later I will use the sklearn toolkit, but in this section **I will develop my own code** to do the Lasso regression.

### Ordinary Least Squares Regression

First, I'll just perform normal linear regression.

I'll use a toy design matrix & labels to use to check that my code works. I'll also specify a weight vector too, for testing.

In [2]:
X = np.array([[0.0,0],[1,3],[2.2,3]])
y = np.array([[0.0,1,2]]).T
w = np.array([[1.0,2]]).T

In [23]:
w

array([[1.],
       [2.]])

In [4]:
y

array([[0.],
       [1.],
       [2.]])

### Prediction Function 

The first task is to write a function to make predictions. I write the prediction function to get the predictions for all the points.

In [24]:
def prediction(X,w):
    preds = X@w
    return preds

In [25]:
# checking whether I've written the right function
np.all(prediction(X,w)==np.array([[0. , 7. , 8.2]]).T) #Should return 'True'

True

###  Objective Function 

Now I need to write a function that returns the 'error'.
I'll just do normal Ordinary Least Squares with linear regression, so the cost function for that is:

$$E = \sum_{i=1}^N \big(y_i-f(\mathbf{x}_i,\mathbf{w}) \big)^2$$

Where $E$ is the error, $N$ the number of points, $y_i$ is one of the labels, $\mathbf{x}_i$ is the input for that label, $\mathbf{w}$ is the weight vector. $f$ is the prediction function I've already written.

In [26]:
def objective(X,y,w):
    """
    Computes the sum squared error (to perform OLS linear regression).
    """
    
    error = (y.T-w.T@X.T)@(y-X@w)
    
    return error

In [27]:
#I have used this code to check if I've written the right function or not
objective(X,y,w)==74.44

array([[ True]])

In [28]:
objective(X,y,w)

array([[74.44]])

###  Objective Function Gradient 

Now I have to find the derivative of the objective wrt the parameter vector. I've already derived this using vectors and matrices, so the gradient of the error function (for linear regression) is:

$$\frac{\partial E}{\partial \mathbf{w}} = 2 X^\top X \mathbf{w} - 2 X^\top y$$

In [29]:
def objective_derivative(X,y,w):
    """
    Computes the derivative of the sum squared error, wrt the parameters.
    """
    
    grad = 2*X.T@X@w-2*X.T@y
    
    return grad
objective_derivative(X,y,w)

array([[39.28],
       [73.2 ]])

To check my gradient function is correct, I can estimate the gradient numerically:

In [30]:
def numerical_objective_derivative(X,y,w):
    """
    Computes a numerical approximation to the derivative of the sum squared error, wrt the parameters.
    """
    g = np.zeros_like(w)
    for i,wi in enumerate(w):
        d = np.zeros_like(w)
        d[i]=1e-6
        g[i] = (objective(X,y,w+d)-objective(X,y,w-d))/2e-6
    return g

The two gradient vectors should be approximately equal:

In [31]:
objective_derivative(X,y,w),numerical_objective_derivative(X,y,w)

(array([[39.28],
        [73.2 ]]),
 array([[39.28],
        [73.2 ]]))

### Optimise $\mathbf{w}$ to minimise the objective

Now I have used the gradient function that I've written to maximise $\mathbf{w}$ using gradient descent. I've looped 1000 times. At each iteration: I've compute the gradient and subtracted the scaled gradient from the w parameter (scaling by the learning rate, of e.g. 0.01).

In [32]:
def optimise_parameters(X,y,startw):
    """
    Returns the w that minimises the objective.
    """
    #Put answer here
    
    n_iters = 1000
    lr = 0.01
    for i in range(n_iters):
        grad = objective_derivative(X,y,startw)
        startw = startw - lr*grad
    
    return startw

In [33]:
bestw = optimise_parameters(X,y,w)
print(bestw) #print our solution

[[0.8333238]
 [0.0555608]]


I've compared this to the answer provided by sklearn:

In [34]:
from sklearn import linear_model
clf = linear_model.LinearRegression(fit_intercept=False)
clf.fit(X,y)
print(clf.coef_) #matches the value of w we found above, hopefully!

[[0.83333333 0.05555556]]


## Lasso Regression

###  New Objective Function 

I'm now going to regularise the regression using $L_1$ regularisation - i.e. Lasso regression.

So I have written the **new objective function**:

$$E = \frac{1}{2N}\sum_{i=1}^N \big(y_i-f(\mathbf{x}_i,\mathbf{w}) \big)^2 + \alpha \sum_{j=1}^P |w_j|$$

This is similar to before (but the first term is now half the mean squared error, rather than the sum squared error). The second term is the L1 regularisation term.

In [35]:
def objective_lasso(X,y,w,alpha):
    """
    Computes half the mean squared error, with an additional L1 regularising term. alpha controls the level of regularisation.
    """
    
    N=len(X)
    shape = (np.shape(w)[0],1)
    I = np.ones(shape)
    
    error = (1/(2*N))*(y.T-w.T@X.T)@(y-X@w)+alpha*w.T@I
    
    return error

###  The gradient of the lasso regression objective 

The tricky bit the derivative of the objective.

The first part is similar to before. So, with the regularising term, the derivative is:
$$\frac{\partial E}{\partial \mathbf{w}} = \frac{1}{N}(X^\top X \mathbf{w} - X^\top y) + \alpha\;\text{sign}(\mathbf{w})$$

where $\text{sign}(\mathbf{w})$ returns a vector of the same shape as $\mathbf{w}$ with +1 if the element is positive and -1 if it's negative. The `np.sign` method does this.

Note: Differentiating $\mathbf{w}$ wrt. itself should return an $\mathbf{identity}$ vector, having same dimension as w. Hence that identity vector has +1 or -1 depending on the sign of the corresponding elements in the vector w.

In [36]:
def objective_lasso_derivative(X,y,w,alpha):
    """
    Returns the derivative of the Lasso objective function.
    """
    N=len(X)
    gradient = (1/N)*(X.T@X@w-X.T@y)+alpha*np.sign(w)
    return gradient

I've checked it again numerically. The two pairs of parameters should be the same:

In [37]:
def numerical_objective_lasso_derivative(X,y,w,alpha):
    """
    This finds a numerical approximation to the true gradient
    """
    g = np.zeros_like(w)
    for i,wi in enumerate(w):
        d = np.zeros_like(w)
        d[i]=1e-6
        g[i] = (objective_lasso(X,y,w+d,alpha)-objective_lasso(X,y,w-d,alpha))/2e-6
    return g

objective_lasso_derivative(X,y,w,0.1),numerical_objective_lasso_derivative(X,y,w,0.1)

(array([[ 6.64666667],
        [12.3       ]]),
 array([[ 6.64666667],
        [12.3       ]]))

### Optimise $\mathbf{w}$ to minimise the Lasso objective 

As before I need to optimise to find the optimum value of $\mathbf{w}$, for this Lasso objective.
We need to loop lots of times (e.g. 5000). At each iteration: I've computed the gradient and subtracted the scaled gradient from the w parameter (scaling it by the learning rate, of e.g. 0.05).

In [38]:
def optimise_parameters_lasso(X,y,startw):
    """
    Returns the w that minimises the Lasso objective.
    """
    n_iters = 5000
    lr = 0.05
    alpha = 0.1
    for i in range(n_iters):
        grad = objective_lasso_derivative(X,y,startw,alpha)
        startw = startw - lr*grad
    return startw


optimise_parameters_lasso(X,y,w)

array([[0.63888889],
       [0.14259259]])

I've checked against the sklearn method:

In [39]:
clf = linear_model.Lasso(alpha=0.1,fit_intercept=False)
clf.fit(X,y)
print(clf.coef_)

[0.63931263 0.1423666 ]


The above result is approximately same as the one I computed.

# Back to air pollution

###  One-hot-encoding

One of the columns isn't numerical, but instead is a string type: The wind direction. The best way to deal with this is one-hot-encoding.

The following steps have been followed:

1. Make the one-hot encoding table using the code above.
2. Delete the `wd` column from our table (hint: you did this earlier for other columns).
3. Join the one hot data to the table. To do this use something like `dataframe1.join(dataframe2)`.



In [40]:
def add_wd_onehot(df):
    """Add new one-hot encoding set of columns, removes the old column it's made from. Returns new dataframe."""
    # Get one hot encoding of columns 'vehicleType'
    
    enc_df = pd.get_dummies(df.wd, prefix = 'direction') # applying get_dummies on the 'direction' feature
    
    # Drop column as it is now encoded
    drop = ['wd']
    df = df.drop(drop, axis=1)

    # Join the encoded df
    df = df.join(enc_df)
    
    return df

#you could use this code to see if it's worked?
train_df_wdencoded = add_wd_onehot(train_df)
train_df_wdencoded

,PM2.5,hour,TEMP,PRES,DEWP,RAIN,WSPM,direction_E,direction_ENE,direction_ESE,...,direction_NNW,direction_NW,direction_S,direction_SE,direction_SSE,direction_SSW,direction_SW,direction_W,direction_WNW,direction_WSW
No,,,,,,,,,,,,,,,,,,,,,
1,9.0,0,0.3,1021.9,-19.0,0.0,2.0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,11.0,1,-0.1,1022.4,-19.3,0.0,4.4,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,8.0,2,-0.6,1022.6,-19.7,0.0,4.7,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,8.0,3,-0.7,1023.5,-20.9,0.0,2.6,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5,8.0,4,-0.9,1024.1,-21.7,0.0,2.5,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29761,145.0,0,27.7,997.1,24.3,0.0,0.9,0,0,0,...,0,0,0,0,0,1,0,0,0,0
29762,139.0,1,26.9,996.6,24.6,0.0,0.8,1,0,0,...,0,0,0,0,0,0,0,0,0,0
29763,128.0,2,26.6,996.3,24.8,0.0,0.4,0,0,0,...,0,0,0,0,0,0,0,0,0,0


###  Normalise the data 

Now we need to normalise the data. Normalising ensures faster convergence in order to get the optimal parameters.

In [85]:
train_df_wdencoded.iloc[:,0:].std()[0]

82.43162771002811

In [86]:
def normalise(df):
    """
    Returns a new dataframe in which all but the PM2.5 columns are normalised (i.e. have a mean of zero and standard deviation of 1)
    """
    df = df.copy()
    ncols = len(df.columns)
    c = -1
   # mean = df.iloc[:,0:].mean()
    #sd =    df.iloc[:,0:].std()
    for cols in df.columns:
        c+=1
        if(c==0): 
            continue
        else:
            
            df[cols] = (df[cols]-df[cols].mean())/df[cols].std() #standardising every column of the dataframe other than the target column
                
    
    return df

train_df_preprocessed = normalise(add_wd_onehot(train_df))
test_df_preprocessed = normalise(add_wd_onehot(test_df))

In [88]:
train_df_preprocessed.iloc[:,0:] # As we can see, all the columns other than the target column has been normalised

,PM2.5,hour,TEMP,PRES,DEWP,RAIN,WSPM,direction_E,direction_ENE,direction_ESE,...,direction_NNW,direction_NW,direction_S,direction_SE,direction_SSE,direction_SSW,direction_SW,direction_W,direction_WNW,direction_WSW
No,,,,,,,,,,,,,,,,,,,,,
1,9.0,-1.657934,-1.244560,1.080123,-1.612174,-0.084705,0.194075,-0.294441,-0.301269,-0.227157,...,-0.18176,-0.279380,-0.214772,-0.184064,-0.172276,-0.286033,-0.332933,-0.292856,3.325914,-0.293132
2,11.0,-1.513617,-1.280183,1.128020,-1.634213,-0.084705,2.157161,-0.294441,-0.301269,-0.227157,...,-0.18176,-0.279380,-0.214772,-0.184064,-0.172276,-0.286033,-0.332933,-0.292856,3.325914,-0.293132
3,8.0,-1.369301,-1.324712,1.147178,-1.663600,-0.084705,2.402547,-0.294441,-0.301269,-0.227157,...,-0.18176,-0.279380,-0.214772,-0.184064,-0.172276,-0.286033,-0.332933,-0.292856,3.325914,-0.293132
4,8.0,-1.224984,-1.333617,1.233393,-1.751759,-0.084705,0.684846,-0.294441,-0.301269,-0.227157,...,-0.18176,3.579237,-0.214772,-0.184064,-0.172276,-0.286033,-0.332933,-0.292856,-0.300659,-0.293132
5,8.0,-1.080667,-1.351429,1.290869,-1.810532,-0.084705,0.603051,-0.294441,-0.301269,-0.227157,...,-0.18176,-0.279380,-0.214772,-0.184064,-0.172276,-0.286033,-0.332933,-0.292856,3.325914,-0.293132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29761,145.0,-1.657934,1.195599,-1.295554,1.568912,-0.084705,-0.705673,-0.294441,-0.301269,-0.227157,...,-0.18176,-0.279380,-0.214772,-0.184064,-0.172276,3.495984,-0.332933,-0.292856,-0.300659,-0.293132
29762,139.0,-1.513617,1.124354,-1.343451,1.590952,-0.084705,-0.787468,3.396145,-0.301269,-0.227157,...,-0.18176,-0.279380,-0.214772,-0.184064,-0.172276,-0.286033,-0.332933,-0.292856,-0.300659,-0.293132
29763,128.0,-1.369301,1.097637,-1.372189,1.605645,-0.084705,-1.114649,-0.294441,-0.301269,-0.227157,...,-0.18176,-0.279380,-0.214772,-0.184064,-0.172276,-0.286033,-0.332933,-0.292856,-0.300659,-0.293132


In [89]:
test_df_preprocessed.head(5)

,PM2.5,hour,TEMP,PRES,DEWP,RAIN,WSPM,direction_E,direction_ENE,direction_ESE,...,direction_NNW,direction_NW,direction_S,direction_SE,direction_SSE,direction_SSW,direction_SW,direction_W,direction_WNW,direction_WSW
No,,,,,,,,,,,,,,,,,,,,,
29766,119.0,-0.941233,1.290070,-2.011871,1.654614,-0.067506,-1.056180,3.277182,-0.367482,-0.253239,...,-0.218796,-0.288404,-0.194767,-0.244875,-0.180899,-0.195304,-0.306222,-0.147823,-0.147134,-0.192062
29767,128.0,-0.796603,1.315659,-1.961072,1.675583,-0.067506,-1.056180,3.277182,-0.367482,-0.253239,...,-0.218796,-0.288404,-0.194767,-0.244875,-0.180899,-0.195304,-0.306222,-0.147823,-0.147134,-0.192062
29768,116.0,-0.651973,1.358307,-1.930592,1.696552,-0.067506,-1.056180,3.277182,-0.367482,-0.253239,...,-0.218796,-0.288404,-0.194767,-0.244875,-0.180899,-0.195304,-0.306222,-0.147823,-0.147134,-0.192062
29769,123.0,-0.507343,1.418014,-1.950912,1.675583,-0.067506,-0.597200,-0.305081,-0.367482,-0.253239,...,-0.218796,-0.288404,-0.194767,-0.244875,-0.180899,-0.195304,-0.306222,-0.147823,-0.147134,-0.192062
29770,148.0,-0.362714,1.460663,-1.940752,1.696552,-0.067506,-0.505404,-0.305081,-0.367482,-0.253239,...,-0.218796,-0.288404,-0.194767,-0.244875,-0.180899,-0.195304,-0.306222,-0.147823,-0.147134,-0.192062


Here I've put the training and test inputs (X) and outputs (y) into four variables:

In [90]:
# Splitiing the training dataframe into inputs X and output Y
X = train_df_preprocessed.iloc[:,1:].to_numpy() 
y = train_df_preprocessed.iloc[:,0].to_numpy()

# Splitiing the test dataframe into inputs Xtest and output ytest
Xtest = test_df_preprocessed.iloc[:,1:].to_numpy()
ytest = test_df_preprocessed.iloc[:,0].to_numpy()

I get the predictions by Lasso regression model from the sklearn.

### Finding the RMSE of the Lasso regressor predictions 

Next I compute the **RMSE** of the predictions for (a) the training data and (b) the test data.

In [91]:
X.shape[1]

22

In [92]:
clf = linear_model.Lasso(alpha=0.25,fit_intercept=True,selection='random') #defining our model
clf.fit(X,y) #fitting it on the training dataframe

Lasso(alpha=0.25, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='random', tol=0.0001, warm_start=False)

In [93]:
#Getting the predictions for training data
y_train_hat = clf.predict(X)

#Getting the predictions for test data
y_test_hat = clf.predict(Xtest)

In [94]:
#Put answer here

#Computing the RMSE for train and test predictions made above
e1 = y_train_hat-y
e2 = y_test_hat-ytest
rmse_lasso_train = np.sqrt(np.mean(e1*e1))
rmse_lasso_test = np.sqrt(np.mean(e2*e2))

In [95]:
rmse_lasso_train, rmse_lasso_test 

(72.4176436233933, 88.86298458492212)

I've compared this to the standard deviation of the data, the model has performed better than that!

In [96]:
np.std(y), np.std(ytest) #Our model performs better than the standard deviation of the data

(82.43021338817387, 103.25109103178704)

###  Random Forest 

The final step is to use a random forest regressor.

If we use the default random forest regressor, we find we may get considerable over-fitting. So we need to explore different parameters. I've used a cross-validated grid search over the parameters:

- max_features: The number of features to consider when looking for the best split (i.e. controls subsampling), *from 1 to the number of features* in 4 steps 
- n_estimators: The number of trees in the forest, from 10 to 100 in 4 steps.
- max_samples : the number of samples to draw from to train each base estimator, from 0.1 to 0.9 in 4 steps.

I've used `GridSearchCV`.


In [97]:
#Put answer here:
#1. Create a grid of parameter values for n_estimators, max_features and max_samples,

parameters = {
    'n_estimators' : np.linspace(10,100,4).astype(int),
    'max_features' : np.linspace(1,X.shape[1],4).astype(int),
    'max_samples'  : np.linspace(0.1,0.9,4)
}


#2. Create a GridSearchCV object, using the random forest regressor:

regressor = RandomForestRegressor(random_state = 0)
grid_regression = GridSearchCV(regressor, parameters,cv=3,scoring='neg_mean_squared_error')

#Note: Because there is so much training data, using the full dataset takes too long. So here we'll just use 10%
np.random.seed(42)
idx = np.sort(np.random.choice(len(X), size=int(len(X)*0.1), replace=False))
#3. Fit to training data in (the subset of) X and y
grid_regression.fit(X[idx,:],y[idx])

GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=0,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jobs=

Here I print the best parameters from the grid search (on the training/validation cross-validation run):

In [98]:
#To obtain the best parameters using grid search and display them
best_n_estimators = grid_regression.best_params_['n_estimators']
best_max_features = grid_regression.best_params_['max_features']
best_max_samples = grid_regression.best_params_['max_samples']
print('Best n_estimator', best_n_estimators)
print('Best max features', best_max_features)
print('Best max samples', best_max_samples)

Best n_estimator 100
Best max features 8
Best max samples 0.6333333333333333


###  RMSE for the Random Forest Regressor

Finally I compute the RMSE for the training and test data:

In [99]:
#getting the training and test predictions for RF using grid search
y_train_hat = grid_regression.predict(X)
y_test_hat = grid_regression.predict(Xtest)

In [100]:

# Computing the rmse for training and test predictions made above
e1 = y_train_hat-y
e2 = y_test_hat-ytest
rmse_rf_train = np.sqrt(np.mean(e1*e1))
rmse_rf_test = np.sqrt(np.mean(e2*e2))


In [101]:
rmse_rf_train, rmse_rf_test

(63.22275111746609, 82.99472812112394)

Again I have compared this to the standard deviations for the two sets of data.

In [103]:
np.std(y), np.std(ytest) # We can see that our model has performed better than the standard deviatons of the two sets of data

(82.43021338817387, 103.25109103178704)

### Useful Observation:


We can see that the random forest model performed better than the lasso reegression model. This is evident from the RMSE values displayed above for both the models

## References :

1. Module Lecture Notes
2. Module lab sheets
3. Dutta, A. (2019). Random Forest Regression in Python - GeeksforGeeks. [online] GeeksforGeeks. Available at: https://www.geeksforgeeks.org/random-forest-regression-in-python/.

4. Zach (2020). Introduction to Lasso Regression. [online] Statology. Available at: https://www.statology.org/lasso-regression/.

